## Learning Objectives
In this notebook, you will learn how to Use the pruned and retrained model in "classification.ipynb" notebook which was exported as a .etlt file for deployment to DeepStream. First, You need to run the "classification.ipynb" notebook using quick deploy option or inside tao-toolkit:4.0.0-tf2.9.1 container to create the export file for this model. Then, Go through this notebook. 

## TAO Deploy Container
Using the tao-deploy container, you can generate a TensorRT engine and verify the correctness of the generated through evaluate and inference.

The tao-deploy produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please run tao-deploy command which will instantiate a deploy container, with the exported .etlt file on your target device. The tao-deploy container only works for x86, with discrete NVIDIA GPU's.
 
**Use the following commands to run this notebook inside the deploy container:**

```
docker run -it --rm --gpus all -v /path/to/notebooks:/workspace/tao --net=host nvcr.io/nvidia/tao/tao-toolkit:4.0.0-tao-toolkit:4.0.0-deploy /bin/bash

pip3 install notebook

pip3 install jupyterlab

cd /workspace

jupyter notebook --ip 0.0.0.0 --port 8888 --allow-root

```

For the jetson devices, please download the tao-converter for jetson and refer to here for more details.

If you choose to integrate your model into deepstream directly, you may do so by simply copying the exported .etlt file along with the calibration cache to the target device and updating the spec file that configures the gst-nvinfer element to point to this newly exported model. Usually this file is called config_infer_primary.txt for detection models and config_infer_secondary_*.txt for classification models.


### Table of Contents
This notebook shows an example use case for classification using the Train Adapt Optimize (TAO) Toolkit.

1. [Set up env variables](#head-1)
2. [Prepare dataset](#head-2)
    1. [Split the dataset into train/test/val](#head-2-1)
    2. [Installing NGC CLI](#head-2-2)

3. [Deploy](#head-3)
4. [Verify the deployed model](#head-4)



## 1. Set up env variables <a class="anchor" id="head-0"></a>
When using the purpose-built pretrained models from NGC, please make sure to set the `$KEY` environment variable to the key as mentioned in the model overview. Failing to do so, can lead to errors when trying to load them as pretrained models.

The following notebook requires the user to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the users workspace. Please note that the dataset to run this notebook is expected to reside in the `$LOCAL_PROJECT_DIR/data`, while the TAO experiment generated collaterals will be output to `$LOCAL_PROJECT_DIR/classification_tf2`. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

*Note: Please make sure to remove any stray artifacts/files from the `$USER_EXPERIMENT_DIR` or `$DATA_DOWNLOAD_DIR` paths as mentioned below, that may have been generated from previous experiments. Having checkpoint files etc may interfere with creating a training graph for a new experiment.*

*Note: This notebook currently is by default set up to run training using 1 GPU. To use more GPU's please update the env variable `$NUM_GPUS` accordingly*

In [1]:
# Setting up env variables for cleaner command line commands.
import os

%env KEY=nvidia_tlt
%env NUM_GPUS=1


# Please define this local project directory that needs to be mapped to the TAO docker session.
# The dataset expected to be present in $LOCAL_PROJECT_DIR/data, while the results for the steps
# in this notebook will be stored at $LOCAL_PROJECT_DIR/classification_tf2
# !PLEASE MAKE SURE TO UPDATE THIS PATH!.
#example: os.environ["LOCAL_PROJECT_DIR"] = '/workspace/tao/tao_launcher_starter_kit/classification_tf2'
os.environ["LOCAL_PROJECT_DIR"] = FIX_ME


os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "data"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "classification_tf2"
)

# The sample spec files are present in the same path as the downloaded samples.
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "tao_voc/specs"
)
#%env SPECS_DIR=/workspace/tao-experiments/classification_tf2/tao_voc/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

env: KEY=nvidia_tlt
env: NUM_GPUS=1
total 20
-rw------- 1 1007 users 1375 Feb  2 18:09 spec_16bit_imgs.yaml
-rw------- 1 1007 users 1402 Feb  2 21:17 spec.yaml
-rw------- 1 1007 users 2494 Feb  3 22:27 spec_retrain_16bit_imgs.yaml
-rw------- 1 1007 users 2046 Feb  4 00:22 spec_retrain_qat.yaml
-rw------- 1 1007 users 2403 Feb  4 00:33 spec_retrain.yaml


## 2. Prepare datasets <a class="anchor" id="head-2"></a>

We will be using the pascal VOC dataset for the tutorial. To find more details please visit 
http://host.robots.ox.ac.uk/pascal/VOC/voc2012/index.html#devkit. Please download the dataset present at http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar to $DATA_DOWNLOAD_DIR.

In [2]:
# Check that file is present
import os
DATA_DIR = os.environ.get('LOCAL_DATA_DIR')
print(DATA_DIR)
if not os.path.isfile(os.path.join(DATA_DIR , 'VOCtrainval_11-May-2012.tar')):
    print('tar file for dataset not found. Please download.')
else:
    print('Found dataset.')

/workspace/tao/tao_launcher_starter_kit/classification_tf2/data
Found dataset.


In [ ]:
# unpack 
!tar -xvf $LOCAL_DATA_DIR/VOCtrainval_11-May-2012.tar -C $LOCAL_DATA_DIR 

In [3]:
# verify
!ls $LOCAL_DATA_DIR/VOCdevkit/VOC2012

Annotations  JPEGImages			 SegmentationClass
ImageSets    JPEGImages_16bit_grayscale  SegmentationObject


### A. Split the dataset into train/val/test <a class="anchor" id="head-2-1"></a>

Pascal VOC Dataset is converted to our format (for classification) and then to train/val/test in the next two blocks.

In [3]:
# install pip requirements
!pip3 install tqdm
!pip3 install matplotlib==3.3.3

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 189.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.0
    Uninstalling matplotlib-3.5.0:
      Successfully uninstalled matplotlib-3.5.0


In [4]:
from os.path import join as join_path
import os
import glob
import re
import shutil

DATA_DIR=os.environ.get('LOCAL_DATA_DIR')
source_dir = join_path(DATA_DIR, "VOCdevkit/VOC2012")
target_dir = join_path(DATA_DIR, "formatted")


suffix = '_trainval.txt'
classes_dir = join_path(source_dir, "ImageSets", "Main")
images_dir = join_path(source_dir, "JPEGImages")
classes_files = glob.glob(classes_dir+"/*"+suffix)
for file in classes_files:
    # get the filename and make output class folder
    classname = os.path.basename(file)
    if classname.endswith(suffix):
        classname = classname[:-len(suffix)]
        target_dir_path = join_path(target_dir, classname)
        if not os.path.exists(target_dir_path):
            os.makedirs(target_dir_path)
    else:
        continue
    print(classname)


    with open(file) as f:
        content = f.readlines()


    for line in content:
        tokens = re.split('\s+', line)
        if tokens[1] == '1':
            # copy this image into target dir_path
            target_file_path = join_path(target_dir_path, tokens[0] + '.jpg')
            src_file_path = join_path(images_dir, tokens[0] + '.jpg')
            shutil.copyfile(src_file_path, target_file_path)

motorbike
car
train
tvmonitor
pottedplant
chair
cow
cat
bus
boat
aeroplane
person
dog
horse
sheep
sofa
bird
bottle
diningtable
bicycle


In [5]:
import os
import glob
import shutil
from random import shuffle
from tqdm import tqdm

DATA_DIR=os.environ.get('LOCAL_DATA_DIR')
SOURCE_DIR=os.path.join(DATA_DIR, 'formatted')
TARGET_DIR=os.path.join(DATA_DIR,'split')
# list dir
print(os.walk(SOURCE_DIR))
dir_list = next(os.walk(SOURCE_DIR))[1]
# for each dir, create a new dir in split
for dir_i in tqdm(dir_list):
        newdir_train = os.path.join(TARGET_DIR, 'train', dir_i)
        newdir_val = os.path.join(TARGET_DIR, 'val', dir_i)
        newdir_test = os.path.join(TARGET_DIR, 'test', dir_i)
        
        if not os.path.exists(newdir_train):
                os.makedirs(newdir_train)
        if not os.path.exists(newdir_val):
                os.makedirs(newdir_val)
        if not os.path.exists(newdir_test):
                os.makedirs(newdir_test)

        img_list = glob.glob(os.path.join(SOURCE_DIR, dir_i, '*.jpg'))
        # shuffle data
        shuffle(img_list)

        for j in range(int(len(img_list)*0.7)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'train', dir_i))

        for j in range(int(len(img_list)*0.7), int(len(img_list)*0.8)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'val', dir_i))
                
        for j in range(int(len(img_list)*0.8), len(img_list)):
                shutil.copy2(img_list[j], os.path.join(TARGET_DIR, 'test', dir_i))
                
print('Done splitting dataset.')

<generator object walk at 0x7f313d7ed120>


100%|███████████████████████████████████████████████████████████████████████████| 20/20 [00:22<00:00,  1.11s/it]

Done splitting dataset.


In [6]:
!ls $LOCAL_DATA_DIR/split/test/cat

2008_000056.jpg  2008_006967.jpg  2009_005037.jpg  2010_003764.jpg
2008_000060.jpg  2008_006973.jpg  2009_005051.jpg  2010_003773.jpg
2008_000096.jpg  2008_007039.jpg  2009_005083.jpg  2010_003781.jpg
2008_000112.jpg  2008_007059.jpg  2009_005158.jpg  2010_003800.jpg
2008_000115.jpg  2008_007082.jpg  2009_005160.jpg  2010_003805.jpg
2008_000116.jpg  2008_007085.jpg  2009_005177.jpg  2010_003818.jpg
2008_000181.jpg  2008_007086.jpg  2009_005211.jpg  2010_003823.jpg
2008_000196.jpg  2008_007106.jpg  2009_005236.jpg  2010_003845.jpg
2008_000222.jpg  2008_007130.jpg  2009_005251.jpg  2010_003861.jpg
2008_000227.jpg  2008_007151.jpg  2009_005299.jpg  2010_003863.jpg
2008_000345.jpg  2008_007176.jpg  2010_000001.jpg  2010_003875.jpg
2008_000358.jpg  2008_007187.jpg  2010_000009.jpg  2010_003898.jpg
2008_000401.jpg  2008_007216.jpg  2010_000043.jpg  2010_003937.jpg
2008_000464.jpg  2008_007256.jpg  2010_000048.jpg  2010_003943.jpg
2008_000619.jpg  2008_007269.jpg  2010_000054.jpg  2010_003950

### B. Installing NGC CLI <a class="anchor" id="head-2-2"></a>

 We will use NGC CLI to get the pre-trained models. For more details, go to ngc.nvidia.com and click the SETUP on the navigation bar.

In [8]:
# Installing NGC CLI on the local machine.
## Download and install
%env CLI=ngccli_cat_linux.zip
!mkdir -p $LOCAL_PROJECT_DIR/ngccli

# Remove any previously existing CLI installations
!rm -rf $LOCAL_PROJECT_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $LOCAL_PROJECT_DIR/ngccli
!unzip -u "$LOCAL_PROJECT_DIR/ngccli/$CLI" -d $LOCAL_PROJECT_DIR/ngccli/
!rm $LOCAL_PROJECT_DIR/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("LOCAL_PROJECT_DIR", ""), os.getenv("PATH", ""))

env: CLI=ngccli_cat_linux.zip
--2023-02-03 23:55:48--  https://ngc.nvidia.com/downloads/ngccli_cat_linux.zip
Resolving ngc.nvidia.com (ngc.nvidia.com)... 13.226.22.68, 13.226.22.85, 13.226.22.36, ...
Connecting to ngc.nvidia.com (ngc.nvidia.com)|13.226.22.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41740281 (40M) [application/zip]
Saving to: ‘/workspace/tao/tao_launcher_starter_kit/classification_tf2/ngccli/ngccli_cat_linux.zip’

ngccli_cat_linux.zi 100%[===================>]  39.81M  99.8MB/s    in 0.4s    

2023-02-03 23:55:48 (99.8 MB/s) - ‘/workspace/tao/tao_launcher_starter_kit/classification_tf2/ngccli/ngccli_cat_linux.zip’ saved [41740281/41740281]

Archive:  /workspace/tao/tao_launcher_starter_kit/classification_tf2/ngccli/ngccli_cat_linux.zip
   creating: /workspace/tao/tao_launcher_starter_kit/classification_tf2/ngccli/ngc-cli/
   creating: /workspace/tao/tao_launcher_starter_kit/classification_tf2/ngccli/ngc-cli/wcwidth/
  inflating: /workspa

In [9]:
!ngc registry model list nvidia/tao/pretrained_classification_tf2:*

+-------+-------+-------+-------+-------+-------+------+-------+-------+
| Versi | Accur | Epoch | Batch | GPU   | Memor | File | Statu | Creat |
| on    | acy   | s     | Size  | Model | y Foo | Size | s     | ed    |
|       |       |       |       |       | tprin |      |       | Date  |
|       |       |       |       |       | t     |      |       |       |
+-------+-------+-------+-------+-------+-------+------+-------+-------+
| effic |       |       |       |       |       | 45.6 | UPLOA | Dec   |
| ientn |       |       |       |       |       | MB   | D_COM | 08,   |
| et_b0 |       |       |       |       |       |      | PLETE | 2022  |
+-------+-------+-------+-------+-------+-------+------+-------+-------+


## 10. Deploy! <a class="anchor" id="head-10"></a>

In [10]:
# Check if etlt model is correctly saved.
!ls -l $LOCAL_EXPERIMENT_DIR/export

total 15852
-rw-r--r-- 1 root root 16228743 Feb  3 23:14 efficientnet-b0.etlt


Using the `tao-deploy` container, you can generate a TensorRT engine and verify the correctness of the generated through evaluate and inference.

The `tao-deploy` produces optimized tensorrt engines for the platform that it resides on. Therefore, to get maximum performance, please run `tao-deploy` command which will instantiate a deploy container, with the exported `.etlt` file on your target device. The `tao-deploy` container only works for x86, with discrete NVIDIA GPU's.

For the jetson devices, please download the tao-converter for jetson and refer to [here](https://docs.nvidia.com/tao/tao-toolkit/text/tensorrt.html#installing-the-tao-converter) for more details.

If you choose to integrate your model into deepstream directly, you may do so by simply copying the exported `.etlt` file along with the calibration cache to the target device and updating the spec file that configures the `gst-nvinfer` element to point to this newly exported model. Usually this file is called `config_infer_primary.txt` for detection models and `config_infer_secondary_*.txt` for classification models.

In [11]:
# Convert to TensorRT engine (FP32).
!classification_tf2 gen_trt_engine -e $LOCAL_SPECS_DIR/spec_retrain.yaml

Loading uff directly from the package source code
python /usr/local/lib/python3.8/dist-packages/nvidia_tao_deploy/cv/classification_tf2/scripts/gen_trt_engine.py  --config-path /workspace/tao/tao_launcher_starter_kit/classification_tf2/tao_voc/specs --config-name spec_retrain.yaml 
Loading uff directly from the package source code
sys:1: UserWarning: 
'spec_retrain.yaml' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/automatic_schema_matching for migration instructions.
<frozen cv.common.hydra.hydra_runner>:86: UserWarning: 
'spec_retrain.yaml' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/automatic_schema_matching for migration instructions.
/usr/local/lib/python3.8/dist-packages/hydra/_internal/hydra.py:119: U

In [12]:
# Convert to TensorRT engine (INT8).
!sed -i "s|fp32|int8|g" $LOCAL_SPECS_DIR/spec_retrain.yaml
!classification_tf2 gen_trt_engine -e $LOCAL_SPECS_DIR/spec_retrain.yaml

Loading uff directly from the package source code
python /usr/local/lib/python3.8/dist-packages/nvidia_tao_deploy/cv/classification_tf2/scripts/gen_trt_engine.py  --config-path /workspace/tao/tao_launcher_starter_kit/classification_tf2/tao_voc/specs --config-name spec_retrain.yaml 
Loading uff directly from the package source code
sys:1: UserWarning: 
'spec_retrain.yaml' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/automatic_schema_matching for migration instructions.
<frozen cv.common.hydra.hydra_runner>:86: UserWarning: 
'spec_retrain.yaml' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/automatic_schema_matching for migration instructions.
/usr/local/lib/python3.8/dist-packages/hydra/_internal/hydra.py:119: U

In [13]:
print('Exported model:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/export/

Exported model:
------------
total 95M
-rw-r--r-- 1 root root 22K Feb  4 00:07 cal.bin
-rw-r--r-- 1 root root 50M Feb  4 00:06 calib.tensorfile
-rw-r--r-- 1 root root 16M Feb  3 23:14 efficientnet-b0.etlt
-rw-r--r-- 1 root root 19M Feb  3 23:58 efficientnet-b0.fp32.engine
-rw-r--r-- 1 root root 12M Feb  4 00:12 efficientnet-b0.int8.engine


In [ ]:
print('Exported QAT model:')
print('------------')
!ls -lh $LOCAL_EXPERIMENT_DIR/export_qat/

## 11. Verify the deployed model <a class="anchor" id="head-11"></a>

Verify the converted engine by visualizing TensorRT inferences.

In [16]:
# Set engine as model_path
!sed -i "s|$LOCAL_EXPERIMENT_DIR/output/weights/$LAST_CHECKPOINT|$LOCAL_EXPERIMENT_DIR/export/efficientnet-b0.fp32.engine|g" $LOCAL_SPECS_DIR/spec_retrain.yaml
!sed -i "s|batch_size: 256|batch_size: 16|g" $LOCAL_SPECS_DIR/spec_retrain.yaml
# Running inference 
!classification_tf2 inference -e $LOCAL_SPECS_DIR/spec_retrain.yaml

Loading uff directly from the package source code
python /usr/local/lib/python3.8/dist-packages/nvidia_tao_deploy/cv/classification_tf2/scripts/inference.py  --config-path /workspace/tao/tao_launcher_starter_kit/classification_tf2/tao_voc/specs --config-name spec_retrain.yaml 
sys:1: UserWarning: 
'spec_retrain.yaml' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/automatic_schema_matching for migration instructions.
<frozen cv.common.hydra.hydra_runner>:86: UserWarning: 
'spec_retrain.yaml' is validated against ConfigStore schema with the same name.
This behavior is deprecated in Hydra 1.1 and will be removed in Hydra 1.2.
See https://hydra.cc/docs/next/upgrades/1.0_to_1.1/automatic_schema_matching for migration instructions.
/usr/local/lib/python3.8/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change

In [17]:
!cat $LOCAL_EXPERIMENT_DIR/output_retrain/result.csv

/workspace/tao/tao_launcher_starter_kit/classification_tf2/data/split/test/aeroplane/2008_000033.jpg,aeroplane,0.98583984
/workspace/tao/tao_launcher_starter_kit/classification_tf2/data/split/test/aeroplane/2008_000037.jpg,aeroplane,0.97558594
/workspace/tao/tao_launcher_starter_kit/classification_tf2/data/split/test/aeroplane/2008_000064.jpg,aeroplane,0.6777344
/workspace/tao/tao_launcher_starter_kit/classification_tf2/data/split/test/aeroplane/2008_000251.jpg,aeroplane,0.4182129
/workspace/tao/tao_launcher_starter_kit/classification_tf2/data/split/test/aeroplane/2008_000291.jpg,aeroplane,0.30322266
/workspace/tao/tao_launcher_starter_kit/classification_tf2/data/split/test/aeroplane/2008_000585.jpg,aeroplane,0.9873047
/workspace/tao/tao_launcher_starter_kit/classification_tf2/data/split/test/aeroplane/2008_000716.jpg,aeroplane,0.9736328
/workspace/tao/tao_launcher_starter_kit/classification_tf2/data/split/test/aeroplane/2008_000756.jpg,aeroplane,0.9584961
/workspace/tao/tao_launcher_s